# Cálculo global do trocador de calor 



Hipótese 1: Tocador concêntrico com escoamento contracorrente

# Propriedades dos fluidos:
    

In [28]:
hot_water_reference_temperature = 35 #temperatura de referência da água
hot_c_p_water = 4178 #J/kg.K capacidade calorífica da água
hot_mu_water = 759*(0.000001) #N.s/m2 Viscosidade da água
hot_conduct_water = 0.621 #W/m.K condutividade da água
hot_water_density = 994.8 #kg/m3
hot_Pr_water = c_p_water*mu_water/conduct_water # Prandtl da água
print(Pr_water)

cold_water_reference_temperature = 22 #temperatura de referência da água
cold_c_p_water = 4180 #J/kg.K capacidade calorífica da água
cold_mu_water = 930*(0.000001) #N.s/m2 Viscosidade da água
cold_conduct_water = 0.608 #W/m.K condutividade da água
cold_water_density = 997.8 #kg/m3
cold_Pr_water = c_p_water*mu_water/conduct_water # Prandtl da água



4.84648


In [29]:

# Condições operacionais

A transferência de calor necessária é obtida de 

$$ q = m^{.}_f c_{p,f} (T_{f,exit} - T_{f, inlet})$$

A área requerida é obtida de 

$$ q = U A F \Delta T_{(lm,CC)} $$

Para escoamento em contra corrente

$$\Delta T_1 = T_{h,inlet} - T_{c,exit } $$
$$\Delta T_2 = T_{h,exit} - T_{c,inlet } $$

$$\Delta T_{(lm,CC)} = \frac{\Delta T_2-\Delta T_1}{ln(\Delta T_2/\Delta T_1)} $$ 

Assim, 

$$ A = \frac{q}{U F \Delta T_{(lm,CC)}} $$ 

$$ L = \frac{A}{\pi D} $$

O Reynolds interno no tubo é

$$ Re_i = \frac{ 4 m^{.}_f}{\pi D \mu}$$

Para escoamento turbulento interno:

$$ Nu_i  = 0.023 Re_i^{4/5}Pr^{0.4} $$

Para escoamento interno em uma tubo em espiral:

$$ Nu_i  = 0.023 Re_i^{0.85}Pr^{0.4} \delta^{0.1}$$

onde $\delta$ é a relação de curvatura, razão entre o raio do tubo e o raio de curvatura do tubo

$$ \delta = \frac{D_i}{2 r_{curv}}$$ 


O coeficiente de transferência de calor interno então é:

$$ h_i = \frac{k}{D} Nu_i $$


Para o lado externo:

No caso de tubo concêntrico:

$$ D_{eo} = \frac{4 A_{flow}}{P_{wetted}} $$

$$ Re_e = \frac{ 4 m^{.}_{hot}}{\pi D_{eo} \mu}$$

Para escoamento turbulento

$$ Nu_e  = 0.023 Re_e^{4/5}Pr^{0.4} $$

Para escoamento laminar, coeficiente é dado pela tabela 8.2 do Incropera:

Para $\frac{D_i}{D_e} = 0.25$, $Nu_e = 4.85$

Escoamento externo em cilindro na direção normal ao eixo circular (Bergman (2014))

$$ Nu_e = C * Re_e^m Pr^{1/3} $$

Para:
$Re_e = 0.4-4$, $C = 0.989$ e $m = 0.33$

$Re_e = 4-40$, $C = 0.911$ e $m = 0.385$

$Re_e = 40-4000 $, $C = 0.683$ e $m = 0.466$

$Re_e = 4000-40000$, $C = 0.193 $ e $m = 0.618$

$Re_e = 40000-400000$, $C = 0.027  $ e $m = 0.805$




O coeficiente de transferência de calor interno então é:

$$ h_e = \frac{k}{D_{eo}} Nu_e $$

O coeficiente global de transferência de calor é dado por:

$$ U = \frac{1}{(1/h_i) + (1/h_e)}$$

In [30]:
import numpy as np
T_cold_exit = 20 #C temperatura desejada na saída
T_cold_inlet = 14 #C temperatura de entrada da água
cold_water_vol_flow_rate = 8.0 #L/min
cold_water_vol_rate = (cold_water_vol_flow_rate/60.0)*0.001 #m3/s
cold_water_flow_rate = cold_water_vol_rate * cold_water_density

T_hot_water_inlet = 30 #C temperatura de entrada da água quente

hot_water_vol_flow_rate = 8.0 #L/min
hot_water_vol_rate = (hot_water_vol_flow_rate/60.0)*0.001 #m3/s
hot_water_flow_rate = hot_water_vol_rate * hot_water_density

internal_diameter = 0.0254/2.0 #
external_diameter = 0.0254
tank_diameter = 0.15 #m, diâmetro do vaso
correction_factor = 1.0 
curv_radius = 0.1 #m - raio de curvatura

print(np.pi)

3.141592653589793


In [39]:
# concêntrica = 1
# serpentina = 2
type = 2

# Transferência de calor necessária no trocador é:
q_cold_water = cold_water_flow_rate * cold_c_p_water * (T_cold_exit-T_cold_inlet)

print("Calor necessário para a troca = ", q_cold_water, "W")

T_hot_water_exit = T_hot_water_inlet - q_cold_water/(hot_water_flow_rate*hot_c_p_water)
print("Temperatura de saída água quente = ", T_hot_water_exit)

if type==1:
    annular_area = (np.pi * (external_diameter*external_diameter) / 4) - (np.pi * (internal_diameter*internal_diameter) / 4)
    wetted_perimeter = np.pi * (external_diameter+internal_diameter)
    outer_hidraul_diameter = 4*annular_area/wetted_perimeter
    print("D_oe = ", outer_hidraul_diameter)
        
    Re_i = 4.0 * cold_water_flow_rate / (np.pi * internal_diameter * mu_water )
    print("Re_int = ", Re_i)
    Nu_i = 0.023 * np.power(Re_i, (4/5.0)) * np.power(Pr_water, (0.4))
    print("Nu_int = ", Nu_i)
    
    Re_e = 4.0 * hot_water_flow_rate / (np.pi * outer_hidraul_diameter * mu_water )
    print("Re_ext = ", Re_e)
    Nu_e_turb = 0.023 * np.power(Re_e, (4/5.0)) * np.power(Pr_water, (0.4))
        
    if(Re_e>3000.0):
        Nu_e = Nu_e_turb
    else:
        Nu_e = 4.85
    print("Nu_ext = ", Nu_e)

if type==2:
    curv_ratio = internal_diameter/(2* curv_radius)
    Re_i = 4.0 * cold_water_flow_rate / (np.pi * internal_diameter * mu_water )
    print("Re_int = ", Re_i)
    Nu_i = 0.023 * np.power(Re_i, (0.85)) * np.power(Pr_water, (0.4)) * np.power(curv_ratio, (0.1))
    print("Nu_int = ", Nu_i)

    flow_velocity = hot_water_vol_rate/(np.pi * tank_diameter*tank_diameter/4.0)
    
    Re_e = hot_water_density * flow_velocity * tank_diameter/ hot_mu_water
    
    if (Re_e> 0.4 and Re_e <= 4.0):
        C = 0.989
        m = 0.33
        
    elif(Re_e> 4.0 and Re_e <= 40):
        C = 0.911
        m = 0.385
            
    elif(Re_e> 40 and Re_e <= 4000):
        C = 0.683
        m = 0.466
        
    elif(Re_e> 4000 and Re_e <= 40000):
        C = 0.193
        m =  0.618

    elif(Re_e> 4000 and Re_e <= 40000):
        C = 0.027
        m =  0.805

    print("Re_e = ", Re_e)
    print("Nu_ext = ", Nu_e)
    Nu_e= C * np.power(Re_e, m)* np.power(hot_Pr_water,1/3.0)

h_i = conduct_water * Nu_i/internal_diameter
print("h_int = ", h_i)

h_e = conduct_water * Nu_e/outer_hidraul_diameter
print("h_ext = ", h_e)

U = 1.0/((1/h_i)+(1/h_e))
print("U = ", U)


delta_T1 = T_hot_water_inlet-T_cold_exit
delta_T2 = T_hot_water_exit - T_cold_inlet

delta_T_log = (delta_T2-delta_T1)/(np.log(delta_T2/delta_T1))
print("Delta T_lm = ",delta_T_log)

A = q_cold_water / (U *correction_factor* delta_T_log )

length = A / (np.pi * internal_diameter)

print("Comprimento necessário de troca = ", length, "m")

Calor necessário para a troca =  3336.6431999999995 W
Temperatura de saída água quente =  23.97902506148295
Re_int =  18397.153302369377
Nu_int =  138.424770362
Re_e =  1483.3771911609308
Nu_ext =  34.7295713282
h_int =  6812.2426359
h_ext =  1709.13244726
U =  1366.3317028
Delta T_lm =  9.98950886066
Comprimento necessário de troca =  6.12711304263 m
